## Действительно ли помогают неразмеченные данные?

Частичное обучение (semi-supervised learning) предлагает методы работы с выборками, в которых лишь для части объектов известны ответы. В статьях утверждается, что добавление неразмеченных данных позволяет повысить качество работы — давайте выясним, так ли это!

Наверное, проще всего добыть неразмеченные примеры, если речь идёт о работе с текстами или изображениями. Остановимся на текстах.

Будем работать с данными из соревнования Predict closed questions on Stack Overflow: https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/data

Нас будет интересовать файл train-sample.csv — загрузите его. Будем решать бинарную задачу: отнесём объект к классу 1, если `OpenStatus == 'open'`, и к классу 0 иначе.

**Задание 1. (5 баллов)**

Загрузите данные и подготовьте выборку. В качестве признаков возьмите TF-IDF по BodyMarkdown с `min_df=10`; про целевую переменную написано выше. Выделите тестовую выборку из 5000 объектов.

In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import SelfTrainingClassifier, LabelPropagation, LabelSpreading
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from scipy.sparse import vstack, csr_matrix
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('train-sample.csv')

df['target'] = (df['OpenStatus'] == 'open').astype(int)
print(f"Распределение классов: 0: {sum(df['target'] == 0)}, 1: {sum(df['target'] == 1)}")

tfidf = TfidfVectorizer(min_df=10, max_features=5000)
X = tfidf.fit_transform(df['BodyMarkdown'].fillna(''))
y = df['target']
print(f"Размер матрицы признаков: {X.shape}")

X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=5000, random_state=42, stratify=y
)
print(f"Обучающая выборка: {X_train_full.shape[0]}, Тестовая: {X_test.shape[0]}")

labeled_size = 500
X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(
    X_train_full, y_train_full, train_size=labeled_size, random_state=42, stratify=y_train_full
)
print(f"Размеченных: {X_labeled.shape[0]}, Неразмеченных: {X_unlabeled.shape[0]}")

Распределение классов: 0: 70136, 1: 70136
Размер матрицы признаков: (140272, 5000)
Обучающая выборка: 135272, Тестовая: 5000
Размеченных: 500, Неразмеченных: 134772


Нас будут интересовать качество (AUC-ROC) в четырёх следующих постановках:
1. Модель обучается только на размеченных данных.
2. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть не пересекается с тестовой выборкой.
3. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть совпадает с тестовой выборкой.
4. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть включает в себя тестовую выборку.

**Задание 1. (5 баллов)**

Проведите эксперименты и сделайте выводы для любого из методов пакета `sklearn.semi_supervised` и для логистической регрессии

In [15]:
def evaluate(model, X_train, y_train, name):
    model.fit(X_train, y_train)
    pred = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, pred)
    print(f"{name}: AUC-ROC = {auc:.6f}")
    return auc

print("\nОбучение только на размеченных данных:")
lr_baseline = LogisticRegression(random_state=42, max_iter=1000)
auc_labeled = evaluate(lr_baseline, X_labeled, y_labeled, "Logistic Regression (только размеченные)")

print("\nSelf-training с неразмеченными данными (не тестовые):")
X_st1 = vstack([X_labeled, X_unlabeled])
y_st1 = np.concatenate([y_labeled, [-1] * X_unlabeled.shape[0]])

base_lr = LogisticRegression(random_state=42, max_iter=1000)
st_model1 = SelfTrainingClassifier(base_lr, threshold=0.75, criterion='threshold')
st_model1.fit(X_st1, y_st1)
pred_st1 = st_model1.predict_proba(X_test)[:, 1]
auc_st1 = roc_auc_score(y_test, pred_st1)
print(f"Self-training (не тестовые): AUC-ROC = {auc_st1:.6f}")

print("\nSelf-training с неразмеченными данными (совпадают с тестом):")
X_st2 = vstack([X_labeled, X_test])
y_st2 = np.concatenate([y_labeled, [-1] * X_test.shape[0]])

st_model2 = SelfTrainingClassifier(base_lr, threshold=0.75, criterion='threshold')
st_model2.fit(X_st2, y_st2)
pred_st2 = st_model2.predict_proba(X_test)[:, 1]
auc_st2 = roc_auc_score(y_test, pred_st2)
print(f"Self-training (тест как неразмеченные): AUC-ROC = {auc_st2:.6f}")

print("\nSelf-training с неразмеченными данными (включают тест):")
X_st3 = vstack([X_labeled, X_unlabeled, X_test])
y_st3 = np.concatenate([y_labeled, [-1] * (X_unlabeled.shape[0] + X_test.shape[0])])

st_model3 = SelfTrainingClassifier(base_lr, threshold=0.75, criterion='threshold')
st_model3.fit(X_st3, y_st3)
pred_st3 = st_model3.predict_proba(X_test)[:, 1]
auc_st3 = roc_auc_score(y_test, pred_st3)
print(f"Self-training (включают тест): AUC-ROC = {auc_st3:.6f}")

print()
results_block2 = pd.DataFrame({
    'Метод': ['Только размеченные', 'Self-training (не тестовые)', 
              'Self-training (тест как неразмеченные)', 'Self-training (включают тест)'],
    'AUC-ROC': [auc_labeled, auc_st1, auc_st2, auc_st3]
})
print(results_block2.to_string(index=False))


Обучение только на размеченных данных:
Logistic Regression (только размеченные): AUC-ROC = 0.742353

Self-training с неразмеченными данными (не тестовые):
Self-training (не тестовые): AUC-ROC = 0.741349

Self-training с неразмеченными данными (совпадают с тестом):
Self-training (тест как неразмеченные): AUC-ROC = 0.747201

Self-training с неразмеченными данными (включают тест):
Self-training (включают тест): AUC-ROC = 0.742052

                                 Метод  AUC-ROC
                    Только размеченные 0.742353
           Self-training (не тестовые) 0.741349
Self-training (тест как неразмеченные) 0.747201
         Self-training (включают тест) 0.742052


### Self-train

Обучаем на размеченной части, предсказываем неразмеченную, потом обучаем на всех, и предсказываем неразмеченную, повторяем пока не сойдёмся в предскзааниях неразмеченной части

In [16]:
print("\nLogistic Regression (базовый):")
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(X_labeled, y_labeled)
pred_lr = lr.predict_proba(X_test)[:, 1]
auc_lr = roc_auc_score(y_test, pred_lr)
print(f"AUC-ROC = {auc_lr:.6f}")

print("\nSelf-training Classifier:")
X_self = vstack([X_labeled, X_unlabeled])
y_self = np.concatenate([y_labeled, [-1] * X_unlabeled.shape[0]])

self_training = SelfTrainingClassifier(
    LogisticRegression(random_state=42, max_iter=1000),
    threshold=0.75, criterion='threshold', max_iter=10
)
self_training.fit(X_self, y_self)
pred_self = self_training.predict_proba(X_test)[:, 1]
auc_self = roc_auc_score(y_test, pred_self)
print(f"AUC-ROC = {auc_self:.6f}")
print(f"Количество итераций: {self_training.n_iter_}")

print("\nLabel Propagation:")
X_prop = vstack([X_labeled[:300], X_unlabeled[:700]])
y_prop = np.concatenate([y_labeled[:300], [-1] * 700])

label_prop = LabelPropagation(kernel='knn', n_neighbors=7, max_iter=100)
label_prop.fit(X_prop, y_prop)
pred_prop = label_prop.predict_proba(X_test)[:, 1]
auc_prop = roc_auc_score(y_test, pred_prop)
print(f"AUC-ROC = {auc_prop:.6f}")

print("\nLabel Spreading:")
label_spread = LabelSpreading(kernel='knn', n_neighbors=7, alpha=0.8, max_iter=100)
label_spread.fit(X_prop, y_prop)
pred_spread = label_spread.predict_proba(X_test)[:, 1]
auc_spread = roc_auc_score(y_test, pred_spread)
print(f"AUC-ROC = {auc_spread:.6f}")

print()
results_block3 = pd.DataFrame({
    'Метод': ['Logistic Regression (базовый)', 'Self-training', 'Label Propagation', 'Label Spreading'],
    'AUC-ROC': [auc_lr, auc_self, auc_prop, auc_spread]
})
print(results_block3.to_string(index=False))


Logistic Regression (базовый):
AUC-ROC = 0.742353

Self-training Classifier:
AUC-ROC = 0.741349
Количество итераций: 10

Label Propagation:
AUC-ROC = 0.632486

Label Spreading:
AUC-ROC = 0.646600

                        Метод  AUC-ROC
Logistic Regression (базовый) 0.742353
                Self-training 0.741349
            Label Propagation 0.632486
              Label Spreading 0.646600


In [17]:
def self_training_manual(X_labeled, y_labeled, X_unlabeled, X_test, y_test,
                         threshold=0.75, max_iter=10):
    """Ручная реализация self-training"""
    X_train = X_labeled.copy()
    y_train = y_labeled.copy()
    X_unl = X_unlabeled.copy()
    prev_preds = None
    
    print(f"Начальное состояние: размечено={X_train.shape[0]}, не размечено={X_unl.shape[0]}")
    
    for iteration in range(max_iter):
        model = LogisticRegression(random_state=42, max_iter=1000)
        model.fit(X_train, y_train)

        test_preds = model.predict_proba(X_test)[:, 1]
        if prev_preds is not None:
            diff = np.mean(np.abs(test_preds - prev_preds))
            if diff < 0.001:
                print(f"Сходимость на итерации {iteration + 1}, diff={diff:.6f}")
                break
        prev_preds = test_preds
        
        if X_unl.shape[0] > 0:
            probas = model.predict_proba(X_unl)
            preds = model.predict(X_unl)
            confidence = np.max(probas, axis=1)
            confident = confidence >= threshold
            
            if np.sum(confident) == 0:
                print(f"Нет уверенных предсказаний на итерации {iteration + 1}")
                break
            
            X_conf = X_unl[confident]
            y_conf = preds[confident]
            
            X_train = vstack([X_train, X_conf])
            y_train = np.concatenate([y_train, y_conf])
            X_unl = X_unl[~confident]
            
            print(f"Итерация {iteration + 1}: добавлено {np.sum(confident)} объектов, "
                  f"осталось {X_unl.shape[0]}, ср. уверенность={confidence[confident].mean():.4f}")
        else:
            print("Все объекты размечены")
            break
        
    final = LogisticRegression(random_state=42, max_iter=1000)
    final.fit(X_train, y_train)
    final_preds = final.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, final_preds)
    
    print(f"\nФинальный результат: AUC-ROC={auc:.6f}, размер выборки={X_train.shape[0]}")
    return auc, iteration + 1, X_train.shape[0]

print("\nЭксперимент с порогом threshold = 0.5")
auc_50, iter_50, size_50 = self_training_manual(
    X_labeled, y_labeled, X_unlabeled, X_test, y_test,
    threshold=0.5, max_iter=10
)

print("\nЭксперимент с порогом threshold = 0.75")
auc_75, iter_75, size_75 = self_training_manual(
    X_labeled, y_labeled, X_unlabeled, X_test, y_test,
    threshold=0.75, max_iter=10
)

print("\nЭксперимент с порогом threshold = 0.9")
auc_90, iter_90, size_90 = self_training_manual(
    X_labeled, y_labeled, X_unlabeled, X_test, y_test,
    threshold=0.9, max_iter=10
)

print()
results_block4 = pd.DataFrame({
    'Threshold': [0.5, 0.75, 0.9],
    'Итераций': [iter_50, iter_75, iter_90],
    'Размер выборки': [size_50, size_75, size_90],
    'AUC-ROC': [auc_50, auc_75, auc_90]
})
print(results_block4.to_string(index=False))


Эксперимент с порогом threshold = 0.5
Начальное состояние: размечено=500, не размечено=134772
Итерация 1: добавлено 134772 объектов, осталось 0, ср. уверенность=0.5762
Все объекты размечены

Финальный результат: AUC-ROC=0.746220, размер выборки=135272

Эксперимент с порогом threshold = 0.75
Начальное состояние: размечено=500, не размечено=134772
Итерация 1: добавлено 693 объектов, осталось 134079, ср. уверенность=0.7693
Итерация 2: добавлено 23034 объектов, осталось 111045, ср. уверенность=0.8112
Итерация 3: добавлено 64977 объектов, осталось 46068, ср. уверенность=0.8755
Итерация 4: добавлено 23795 объектов, осталось 22273, ср. уверенность=0.8464
Итерация 5: добавлено 5010 объектов, осталось 17263, ср. уверенность=0.7918
Итерация 6: добавлено 1577 объектов, осталось 15686, ср. уверенность=0.7719
Итерация 7: добавлено 775 объектов, осталось 14911, ср. уверенность=0.7660
Итерация 8: добавлено 600 объектов, осталось 14311, ср. уверенность=0.7642
Итерация 9: добавлено 163 объектов, остал

In [18]:
best_method_idx = np.argmax([auc_st1, auc_self, auc_prop, auc_spread])
methods = ['Self-training (не тест)', 'Self-training (sklearn)', 'Label Propagation', 'Label Spreading']
best_method = methods[best_method_idx]
best_score = max(auc_st1, auc_self, auc_prop, auc_spread)

print(f"""
1. Сравнение с базовой моделью:
   - Базовая Logistic Regression (только размеченные): {auc_labeled:.6f}
   - Self-training с неразмеченными (не тест): {auc_st1:.6f}
   - Улучшение: {auc_st1 - auc_labeled:+.6f}

2. Влияние тестовых данных:
   - Когда тест используется как неразмеченные: {auc_st2:.6f}
   - Когда тест включен в неразмеченные: {auc_st3:.6f}

3. Сравнение semi-supervised методов:
   - Self-training (sklearn): {auc_self:.6f}
   - Label Propagation: {auc_prop:.6f}
   - Label Spreading: {auc_spread:.6f}

4. Влияние порога уверенности в self-training:
   - Порог 0.5: {auc_50:.6f} (добавлено {size_50 - 500} объектов)
   - Порог 0.75: {auc_75:.6f} (добавлено {size_75 - 500} объектов)
   - Порог 0.9: {auc_90:.6f} (добавлено {size_90 - 500} объектов)

5. ОСНОВНЫЕ ВЫВОДЫ:
   - Лучший метод среди всех: {best_method} с ROC-AUC = {best_score:.6f}

""")


1. Сравнение с базовой моделью:
   - Базовая Logistic Regression (только размеченные): 0.742353
   - Self-training с неразмеченными (не тест): 0.741349
   - Улучшение: -0.001004

2. Влияние тестовых данных:
   - Когда тест используется как неразмеченные: 0.747201
   - Когда тест включен в неразмеченные: 0.742052

3. Сравнение semi-supervised методов:
   - Self-training (sklearn): 0.741349
   - Label Propagation: 0.632486
   - Label Spreading: 0.646600

4. Влияние порога уверенности в self-training:
   - Порог 0.5: 0.746220 (добавлено 134772 объектов)
   - Порог 0.75: 0.741349 (добавлено 120710 объектов)
   - Порог 0.9: 0.742353 (добавлено 0 объектов)

5. ОСНОВНЫЕ ВЫВОДЫ:
   - Лучший метод среди всех: Self-training (не тест) с ROC-AUC = 0.741349


